In [1]:
import os
import numpy as np
import pandas as pd
import supervision as sv
from supervision.metrics import MeanAveragePrecision

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

classes= ['CFCBK', 'FCBK', 'Zigzag']

In [2]:
# Configuration files
model_configs = [
{
'backbone': 'resnet50',
'head': 'OrientedRCNN',
'config_file': 'configs/oriented_rcnn/oriented_rcnn_r50_fpn_1x_brickkiln_le90.py',
'checkpoint_folder': 'work_dirs/oriented_rcnn_r50_fpn_1x_brickkiln_le90_first',
'val_dir': '../data/sentinel/test',
'inf_dir': 'results/oriented_rcnn',
'img_height': 128,
'epoch': 12,
},
]

In [8]:
def get_image_names_from_directory(directory):
    """Extracts image names (without extension) from a directory."""
    return {file_name.replace(".txt", "") for file_name in os.listdir(directory) if file_name.endswith(".txt")}

def load_detections(annotations_path, img_names, is_gt=True, confidence_threshold=0):
    """Loads detections only for images that exist in both GT and Predictions."""
    sv_data = []

    for image_id in sorted(img_names):
        file_path = os.path.join(annotations_path, f"{image_id}.txt")
        if not os.path.exists(file_path):  # Ensure file exists before processing
            continue

        xyxy_list = []
        class_ids = []
        scores = []

        with open(file_path, "r") as file:
            lines = file.readlines()

        if not lines:
            detection = sv.Detections(
                xyxy=np.empty((0, 4)),
                class_id=np.empty((0,)),
                confidence=np.empty((0,)),
                # metadata={"image_id": image_id}
            )
            sv_data.append(detection)
            continue
        
        for line in lines:
            data = line.strip().split(" ")
            class_name = data[8]
            class_id = classes.index(class_name) if class_name in classes else -1
            if class_id == -1:
                continue
            polygon = np.array(list(map(float, data[:8]))).reshape(4, 2)  # Convert to (4,2) shape with floats
            score = float(data[9]) if not is_gt else 1.0  # Default confidence for GT is 1.0

            if not is_gt and score < confidence_threshold:
                continue

            # Convert quadrilateral to bounding box (min x, min y, max x, max y)
            x_min, y_min = np.min(polygon, axis=0)
            x_max, y_max = np.max(polygon, axis=0)
            bbox = [x_min, y_min, x_max, y_max]

            # Append to lists
            xyxy_list.append(bbox)
            class_ids.append(class_id)
            scores.append(score)

        # Convert lists into a Supervision Detections object
        detections = sv.Detections(
            xyxy=np.array(xyxy_list),
            class_id=np.array(class_ids),
            confidence=np.array(scores),
            # metadata={"image_id": image_id}
        )

        sv_data.append(detections)

    return sv_data

def get_class_counts(detections_list, num_classes=3):
    """Counts occurrences of each class in ground truth detections."""
    class_counts = np.zeros(num_classes)
    for detections in detections_list:
        unique, counts = np.unique(detections.class_id, return_counts=True)
        for cls, count in zip(unique, counts):
            # print(cls, count)
            class_counts[cls] += count
    return class_counts


In [9]:
index = pd.MultiIndex.from_tuples([], names=["Head", "Backbone", "Epochs"])
result_df = pd.DataFrame(columns=["CFCBK", "FCBK", "Zigzag", "Weighted mAP@50", "mAP@50:95", "mAP@50", "mAP@75", "CA mAP@50:95", "CA mAP@50", "CA mAP@75"], index=index)

In [10]:
confidence_threshold = 0.05

In [ ]:
for model_config in model_configs:
    # Load image names from directories
    backbone = model_config['backbone']
    head = model_config['head']
    GT_PATH = os.path.join(model_config['val_dir'], "annfiles")
    # PREDICTIONS_PATH = os.path.join(model_config['inf_dir'] + f"/epoch_{model_config['epoch']}_supervision_conf_0.01_nms_0.33", "annfiles")
    PREDICTIONS_PATH = os.path.join(model_config['inf_dir'] + f"/epoch_{model_config['epoch']}", "annfiles")
    # PREDICTIONS_PATH = os.path.join(model_config['inf_dir'], "annfiles")
    gt_img_names = get_image_names_from_directory(GT_PATH)
    pred_img_names = get_image_names_from_directory(PREDICTIONS_PATH)
    img_names = gt_img_names.intersection(pred_img_names)
    # base_state = model_config['train']
    # target_state = model_config['test']
    epoch = model_config['epoch']

    # Load GT and Predictions
    gt_data = load_detections(GT_PATH, img_names, is_gt=True)
    pred_data = load_detections(PREDICTIONS_PATH, img_names, is_gt=False, confidence_threshold=confidence_threshold)
    # print(gt_data[0])
    # print(pred_data[0])

    # Print mAP results
    # print(f"\n{model_config['train']} to {model_config['test']} (Epoch {model_config['epoch']}):")
    ## mAP calculation (non-class agnostic)
    mAP_metric = MeanAveragePrecision(class_agnostic=False)
    mAP_result = mAP_metric.update(pred_data, gt_data).compute()
    # print(mAP_result)
    matched_classes = mAP_result.matched_classes.tolist()
    print(f"    Matched classes: {matched_classes}")
    # Extract overall mAP values
    mAP_50_95 = mAP_result.map50_95  # mAP 50:95
    mAP_50 = mAP_result.map50  # mAP 50
    mAP_75 = mAP_result.map75  # mAP 75
    print(f"    mAP 50:95: {mAP_50_95}, mAP 50: {mAP_50}, mAP 75: {mAP_75}")

    # Extract class-wise mAP
    # print(mAP_result.ap_per_class)
    class_wise_mAP = mAP_result.ap_per_class[:, 0].tolist()  # mAP 50:95 per class
    num_classes = 3
    final_class_wise_mAP = [0] * num_classes
    for cls, mAP in zip(matched_classes, class_wise_mAP):
        # print(f"    cls: {cls}, mAP: {mAP}")
        final_class_wise_mAP[cls] = mAP
    print(f"    class_wise_mAP: {final_class_wise_mAP}\n")
    # Calculate weighted mAP
    class_counts = get_class_counts(gt_data, num_classes=num_classes)
    print(f"    class_counts: {class_counts}")
    weights = 1 / np.sum(class_counts)
    weights /= np.sum(weights)
    weighted_mAP_50 = np.sum(np.array(final_class_wise_mAP) * weights)
    print(f"    Weighted mAP 50: {weighted_mAP_50}\n")
    

    # Compute class-agnostic mAP
    mAP_metric_agnostic = MeanAveragePrecision(class_agnostic=True)
    mAP_result_agnostic = mAP_metric_agnostic.update(pred_data, gt_data).compute()
    # Extract class-agnostic mAP values
    mAP_50_95_agnostic = mAP_result_agnostic.map50_95  # mAP 50:95
    mAP_50_agnostic = mAP_result_agnostic.map50  # mAP 50
    mAP_75_agnostic = mAP_result_agnostic.map75  # mAP 75
    print(f"    CA mAP 50:95: {mAP_50_95_agnostic}, CA mAP 50: {mAP_50_agnostic}, CA mAP 75: {mAP_75_agnostic}")

    # Update results dataframe
    result_df.loc[(head, backbone, epoch), :] = [f"{x:.6f}" for x in final_class_wise_mAP + [weighted_mAP_50, mAP_50_95, mAP_50, mAP_75, mAP_50_95_agnostic, mAP_50_agnostic, mAP_75_agnostic]]

    Matched classes: [0, 1, 2]
    mAP 50:95: 0.03741129127496594, mAP 50: 0.13536756016955995, mAP 75: 0.007586505822033612
    class_wise_mAP: [0.0, 0.0, 0.4061026805086798]

    class_counts: [  2.  24. 288.]
    Weighted mAP 50: 0.3724763439060503

    CA mAP 50:95: 0.114840703302865, CA mAP 50: 0.41896378300093207, CA mAP 75: 0.0197977564093893


In [13]:
display(result_df)

,,,CFCBK,FCBK,Zigzag,Weighted mAP@50,mAP@50:95,mAP@50,mAP@75,CA mAP@50:95,CA mAP@50,CA mAP@75
Head,Backbone,Epochs,,,,,,,,,,
OrientedRCNN,resnet50,12,0.000000,0.000000,0.406103,0.372476,0.037411,0.135368,0.007587,0.114841,0.418964,0.019798


Append to save to csv file

In [16]:
append_to_csv = True

if append_to_csv:
    # save the dataframe as csv. if it exists, append to it
    csv_filename = f"mAP_results.csv"
    if os.path.exists(csv_filename):
        result_df.to_csv(csv_filename, mode='a', header=False)
    else:
        result_df.to_csv(csv_filename)

In [18]:
mAP_df = pd.read_csv("mAP_results.csv", index_col=[0,1,2])
display(mAP_df)

,,,CFCBK,FCBK,Zigzag,Weighted mAP@50,mAP@50:95,mAP@50,mAP@75,CA mAP@50:95,CA mAP@50,CA mAP@75
Head,Backbone,Epochs,,,,,,,,,,
OrientedRCNN,resnet50,12,0.0,0.0,0.406103,0.372476,0.037411,0.135368,0.007587,0.114841,0.418964,0.019798
